In [1]:
# Imports
import pandas as pd

def read_dataset(year):
    # Read the database .CSV
    df = pd.read_csv(
        f'https://raw.githubusercontent.com/centraldedados/protecao_civil/master/data/anpc-{year}.csv', 
        sep = ',', 
        on_bad_lines='skip'
    )

    # We replace the "," with "." to facilitate processing
    df['Latitude'] = pd.to_numeric(
        df['Latitude'].str.replace(',', '.')
    )
    df['Longitude'] = pd.to_numeric(
        df['Longitude'].str.replace(',', '.')
    )
    return df

In [2]:
occ = pd.concat([read_dataset(2016),read_dataset(2017),read_dataset(2018)])

/tmp/ipykernel_3430/3896029409.py:6: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(
/tmp/ipykernel_3430/3896029409.py:6: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(


In [3]:
occ['DataOcorrencia'] = pd.to_datetime(occ['DataOcorrencia'], format='%d/%m/%Y %H:%M:%S', errors='coerce') 
occ['DataFechoOperacional'] = pd.to_datetime(occ['DataFechoOperacional'], format='%d/%m/%Y %H:%M:%S', errors='coerce')

occ['Date'] = occ['DataOcorrencia'].dt.date
occ = occ.dropna(subset=["DataOcorrencia", "Latitude"])
occ = occ.loc[occ['NumeroOperacionaisTerrestresEnvolvidos'] < 2000]
occ.loc[occ['Latitude'] > 43, 'Latitude']/=1000
occ.loc[occ['Longitude'] > 0, 'Longitude']*=-1
occ.loc[occ['Longitude'] < -1000, 'Longitude']/=1000

In [4]:
print(occ['Latitude'].min(), occ['Latitude'].max())
print(occ['Longitude'].min(), occ['Longitude'].max())

-8.756731 42.222705
-68.58389 -6.195511


In [5]:
import datetime
occ = occ.loc[(occ['Longitude'] < -6) & (occ['Longitude'] > -10) & (occ['Latitude']<43) & (occ['Latitude'] > 36)]
occ = occ.loc[occ['Date'] >= datetime.date(year=2016,month=5,day=11)]

In [8]:
import numpy as np

data = []
minX, minY, maxX, maxY = occ['Latitude'].min(), occ['Longitude'].min(),  occ['Latitude'].max(), occ['Longitude'].max()
square_size = 0.25

for id, d in occ.groupby(['Date']):
    print("Working on", id)
    grid = []
    for x in np.arange(minX, maxX + square_size,square_size):
        line = []
        for y in np.arange(minY, maxY+ square_size, square_size):
            feature = []
            grid_date_df = d.loc[(d['Latitude'] >= x)& (d['Latitude'] < x+square_size) & (d['Longitude'] >= y) & (d['Longitude'] < y+square_size)]
            for col in ['NumeroMeiosTerrestresEnvolvidos', 'NumeroOperacionaisTerrestresEnvolvidos', 'NumeroMeiosAereosEnvolvidos','NumeroOperacionaisAereosEnvolvidos']:
                aggre = grid_date_df.agg({col : ['sum']})
                aggre.columns = ['F1']
                feature.append(aggre['F1'])
            line.append(feature)
        grid.append(line)
    data.append(grid)

arr_np = np.array(data)
print(arr_np.shape)

/tmp/ipykernel_3430/419219143.py:7: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for id, d in occ.groupby(['Date']):


Working on 2016-05-11
Working on 2016-05-12
Working on 2016-05-13
Working on 2016-05-14
Working on 2016-05-15
Working on 2016-05-16
Working on 2016-05-17
Working on 2016-05-18
Working on 2016-05-19
Working on 2016-05-20
Working on 2016-05-21
Working on 2016-05-22
Working on 2016-05-23
Working on 2016-05-24
Working on 2016-05-25
Working on 2016-05-26
Working on 2016-05-27
Working on 2016-05-28
Working on 2016-05-29
Working on 2016-05-30
Working on 2016-05-31
Working on 2016-06-01
Working on 2016-06-02
Working on 2016-06-03
Working on 2016-06-04
Working on 2016-06-05
Working on 2016-06-06
Working on 2016-06-07
Working on 2016-06-08
Working on 2016-06-09
Working on 2016-06-10
Working on 2016-06-11
Working on 2016-06-12
Working on 2016-06-13
Working on 2016-06-14
Working on 2016-06-15
Working on 2016-06-16
Working on 2016-06-17
Working on 2016-06-18
Working on 2016-06-19
Working on 2016-06-20
Working on 2016-06-21
Working on 2016-06-22
Working on 2016-06-23
Working on 2016-06-24
Working on

In [10]:
with open('data.npy', 'wb') as f:
    np.save(f, arr_np)